In [2]:
from hazm import *
import re 
import pickle 
import math
import sys 
import nltk          
import tkinter as tk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
K=10
import pandas as pd 
import collections
from PyQt5.QtWidgets import QWidget,QScrollArea, QTableWidget, QVBoxLayout,QTableWidgetItem
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
%gui qt5 
stopword = nltk.corpus.stopwords.words('english')
stemmer = PorterStemmer()
lemmatiz = WordNetLemmatizer()

In [3]:
#read dataset 
df = pd.read_csv(r'C:\Users\sepehr\Desktop\presentation\1974_3493_compressed_articles1.csv\articles1.csv')
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [4]:
#preprocess the query 

    
def remove_punct(text):
        text = re.sub('[0-9]+', '', text)
        return text
    
    
    
 
    
def preprocess(data):

    data=  re.sub('https|com|tweet|twitter', '', data)
    
    
    
    data=  re.sub('[,\.!?—]', '', data)
    data=data.replace(" ", " ")
    
   
    # Convert the titles to lowercase
    data =  data.lower()
    
    data = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', data) # remove URLs
    data =  re.sub('@[^\s]+', 'AT_USER', data)
    data =  re.sub(r'#([^\s]+)', r'\1', data)
        
    return data 
     


      
        
def tokenization(text):
    text = re.split('\W+', text)
    return text

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

    


def stemming(text):
    text = [stemmer.stem(word) for word in text]
    return text



def lemmatizer(text):
    #print("new line")
    listof=[]
    for word in text:
      lemi=  lemmatiz.lemmatize(word)
 
            
            
      listof.append(str(lemi))      
    return listof


In [5]:
def word_count(words):
    counts = dict()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts




def tffunc(token,frequency=1):
    tfDic = {}
    counter=word_count(token)
    for word, count in counter.items(): 
            tfDic[word] =1+math.log10(count)
            
            
            tfDic[word]=tfDic[word]*1
            
    return tfDic           

In [6]:
def vectorize(query):
   queryvec= tffunc(query,frequency=1)
    
   return queryvec


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


In [7]:
#champion list and elimation index 
def champion_combine(input1,cham_list):
    listcham=[]
    for word in input1:
        if word in cham_list.keys():
            second_list=list(cham_list[word])
            listcham.extend(x for x in second_list if x not in listcham)
            
    return   listcham

    
def elimination(output,input1,postinglist):
    listofeliminate=[]
    for word in input1:
        if word in postinglist.keys():
            second_list=list(postinglist[word])
            listofeliminate.extend(x for x in second_list if x not in listofeliminate)
            
    return   listofeliminate
    

In [8]:
class MaxHeap: 
  
    def __init__(self, maxsize): 
        self.maxsize = maxsize 
        self.size = 0
        self.Heap = [0]*(self.maxsize + 1) 
        self.Heap[0] = sys.maxsize 
        self.FRONT = 1
  
    def parent(self, pos): 
        return pos//2
    
    def leftChild(self, pos): 
        return 2 * pos 
  

    def rightChild(self, pos): 
        return (2 * pos) + 1
 
    def isLeaf(self, pos): 
        if pos >= (self.size//2) and pos <= self.size: 
            return True
        return False

    def swap(self, fpos, spos): 
        self.Heap[fpos], self.Heap[spos] = self.Heap[spos], self.Heap[fpos] 
  
   
    def maxHeapify(self, pos): 
        if not self.isLeaf(pos): 
            if (self.Heap[pos] < self.Heap[self.leftChild(pos)] or
                self.Heap[pos] < self.Heap[self.rightChild(pos)]): 
  
                if self.Heap[self.leftChild(pos)] > self.Heap[self.rightChild(pos)]: 
                    self.swap(pos, self.leftChild(pos)) 
                    self.maxHeapify(self.leftChild(pos)) 
  
                # Swap with the right child and heapify 
                # the right child 
                else: 
                    self.swap(pos, self.rightChild(pos)) 
                    self.maxHeapify(self.rightChild(pos)) 
  
    # Function to insert a node into the heap 
    def insert(self, element): 
        if self.size >= self.maxsize : 
            return
        self.size+= 1
        self.Heap[self.size] = element 
  
        current = self.size 
  
        while self.Heap[current] > self.Heap[self.parent(current)]: 
            self.swap(current, self.parent(current)) 
            current = self.parent(current) 
  
    # Function to print the contents of the heap 
    def Print(self): 
        for i in range(1, (self.size//2)+1): 
            print(" PARENT : "+str(self.Heap[i])+" LEFT CHILD : "+ 
                               str(self.Heap[2 * i])+" RIGHT CHILD : "+
                               str(self.Heap[2 * i + 1])) 
  
    # Function to remove and return the maximum 
    # element from the heap 
    def extractMax(self): 
  
        popped = self.Heap[self.FRONT] 
        self.Heap[self.FRONT] = self.Heap[self.size] 
        self.size-= 1
        self.maxHeapify(self.FRONT) 
        return popped 
    
    


In [9]:
a_file2 = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_freq.pkl", "rb")
postinglist = pickle.load(a_file2)

In [10]:
import regex as re

ModuleNotFoundError: No module named 'regex'

In [11]:
    
import time 
    
    
always_true_flag=True
flag_champion=False
import PySimpleGUI as sg
    
sg.theme('DarkAmber')# Add a touch of color
    # All the stuff inside your window.
layout = [  [sg.Text('Please enter your query ')],
                [sg.Text('in'), sg.InputText()],
                [sg.Button('Ok'), sg.Button('Cancel'),sg.Button('Champion List')] ]
    
    # Create the Window
window = sg.Window('Window Title', layout)
    # Event Loop to process "events" and get the "values" of the inputs
while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Cancel':	# if user closes window or clicks cancel
            break
        if  event == 'Champion List':
            flag_champion=True
            
        print('You entered ', values[0])
    
    window.close()
a= time.time()
    
    
input1=values[0]



   
input1 =  preprocess(input1)
input1=  remove_punct(input1)
input1 =  tokenization(input1)
print("tokenization done")
input1 =  remove_stopwords(input1)
print("remove_stopwords done")
input1 = stemming(input1)
input1= lemmatizer(input1)
    
a_file = open(r"C:\Users\sepehr\Desktop\presentation\ir\tfidf.pkl", "rb")
output = pickle.load(a_file)
#print("tfidf first news        "+str(output[0]))
a_file2 = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_freq.pkl", "rb")
postinglist = pickle.load(a_file2)
    
    #a_file = open(r"data1.pkl", "rb")
    #output = pickle.load(a_file)
    #print(output[0])

    


query_vec=vectorize(input1)
print("query_vectorize :      "+str(query_vec))
    
    
if(flag_champion==False):
            print("false")
            eliminate = elimination(output,input1,postinglist)
if(flag_champion==True):
            print("true")
            a_file2 = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_cham_final.pkl", "rb")
            champion_dic = pickle.load(a_file2)
            eliminate=champion_combine(input1,champion_dic)
            print(eliminate)
            
        
if(always_true_flag):        
            print([item for item, count in collections.Counter(eliminate).items() if count > 1])
            dict1={} 
            minHeap = MaxHeap(len(eliminate)) 
            for i in range(len(eliminate)):
                cosine = get_cosine(query_vec, output[eliminate[i]])
                minHeap.insert(cosine) 
                dict1[cosine]=eliminate[i]
            listiofoutput=[] 
            dataframefinal=pd.DataFrame()
            for i in range(0,K):
                listiofoutput.append((minHeap.extractMax()))
                                     
                                     
            for i in range(0,K):
               #print(listiofoutput[i])     
            result=pd.DataFrame()                    
        
            for i  in range(0,len(listiofoutput)):
                
                
                if listiofoutput[i] in dict1.keys():
                  dataframefinal=dataframefinal.append(df[dict1[listiofoutput[i]]:dict1[listiofoutput[i]]+1])
                  result=result.append(df[dict1[listiofoutput[i]]:dict1[listiofoutput[i]]+1])
                  print(result)

resultfinal=pd.DataFrame()
resultfinal['Title']=result['title']
resultfinal['Content']=result['content']
resultfinal=resultfinal.reset_index()
b=time.time()
print(str(b-a)+"time")


You entered  stock apple
tokenization done
remove_stopwords done
query_vectorize :      {'stock': 1.0, 'appl': 1.0}
false
[]
0.48652203921857395
0.3870904707417848
0.33190294172861196
0.28315946438180295
0.26535901243537635
0.2557262969762065
0.2500234686359621
0.2451133906232667
0.24465753864666753
0.23302652208801614
       Unnamed: 0     id                                              title  \
46820       49886  69459  Apple bought at least 3 companies in the past ...   

            publication author        date    year  month  url  \
46820  Business Insider    NaN  2016-05-05  2016.0    5.0  NaN   

                                                 content  
46820  ’’ ’Apple buys so many companies that there ar...  
       Unnamed: 0     id                                              title  \
46820       49886  69459  Apple bought at least 3 companies in the past ...   
45889       48864  68278  Apple says government lost access to shooter’s...   

            publication author 

In [13]:
resultfinal["Content"]

0    ’’ ’Apple buys so many companies that there ar...
1    ’Senior Apple executives have accused the San ...
2    ’’ ’The initial public offering freeze could f...
3    ”On August 24, 2011, Tim Cook took over from S...
4    ’BlackRock has upped its stake in some   tech ...
5    ’’ ’This is the big one, folks.’ ’Apple will r...
6     (CNN) CNN’s live coverage on Apple News of th...
7    ’  ’ ’   The battle between the New York Stock...
8    ’’ ’Former US Vice President Al Gore sold ’ ’’...
9    ’Apple shares closed down for an   day on Mond...
Name: Content, dtype: object